In [1]:
%load_ext lab_black
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import glob
import tqdm
import umap
import lightgbm as lgb
import numpy as np

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt

from utils import DATA_SETS, gather_df, prepare_df, model_function

In [2]:
DATA_SETS

['aquifer_auser',
 'water_spring_amiata',
 'aquifer_petrignano',
 'aquifer_doganella',
 'aquifer_luco',
 'river_arno',
 'lake_bilancino',
 'water_spring_lupa',
 'water_spring_madonna_di_canneto']

In [ ]:
import optuna
import json

result_folder = "sim-res-final"

info_file_name = "./{}/info.json".format(result_folder)
if os.path.exists(info_file_name):
    with open(info_file_name) as f:
        info = json.loads(f.read())
else:
    info = []

for dataset in DATA_SETS:
    df = gather_df(dataset, True)
    target_cols = [c for c in df.columns if "target" in c]

    for target_col in target_cols:
        for pred_ahead in [14, 28, 56]:
            # compute the size of location_matrix_W, which is variable per dataset
            n_feats = (
                len([c for c in df.columns if "ws10m_max" in c])
                + len([c for c in df.columns if "rainfall" in c])
                + len([c for c in df.columns if "temperature" in c])
            )

            # this is the optimization function that we are going to hyperoptimize
            def objective(trial, return_dataframes=False):
                location_array_W = []
                for i in range(n_feats):
                    location_array_W.append(trial.suggest_float("W_{}".format(i), 0, 1))
                dfp_val, dfp_test, lgb_model = model_function(
                    dataset,
                    location_array_W,
                    pred_ahead,
                    target_col,
                    extended_data=trial.suggest_categorical(
                        "extended_data", [True, False]
                    ),
                    impute_missing=trial.suggest_categorical(
                        "impute_missing", [True, False]
                    ),
                    do_extract=trial.suggest_categorical("do_extract", [True, False]),
                    shift_features=trial.suggest_categorical(
                        "shift_features", [True, False]
                    ),
                    use_early_stopping=trial.suggest_categorical(
                        "use_early_stopping", [True, False]
                    ),
                    lgb_boosting_type=trial.suggest_categorical(
                        "lgb_boosting_type", ["gbdt", "goss", "dart"]
                    ),
                    lgb_num_leaves=trial.suggest_categorical(
                        "lgb_num_leaves", [10, 31, 50]
                    ),
                    lgb_learning_rate=trial.suggest_uniform(
                        "lgb_learning_rate", 0.01, 0.3
                    ),
                    lgb_max_depth=trial.suggest_int(
                        "lgb_max_depth", -1, 10
                    ),  # -1 here means infinite
                )
                if return_dataframes:
                    return dfp_val, dfp_test
                return np.mean(
                    np.abs(dfp_val.p - dfp_val.y)
                )  # we optimize on the MAE of the validation dataset

            study = optuna.create_study()
            study.optimize(objective, n_trials=100)

            # the outputting dataframe with optimal parameters
            location_array_W_optim = []
            for k, v in study.best_params.items():
                if "W_" in k:
                    location_array_W_optim.append(v)

            dfp_val, dfp_test, lgb_model = model_function(
                dataset,
                location_array_W_optim,
                pred_ahead,
                target_col,
                extended_data=study.best_params["extended_data"],
                impute_missing=study.best_params["impute_missing"],
                do_extract=study.best_params["do_extract"],
                shift_features=study.best_params["shift_features"],
                use_early_stopping=study.best_params["use_early_stopping"],
                lgb_boosting_type=study.best_params["lgb_boosting_type"],
                lgb_num_leaves=study.best_params["lgb_num_leaves"],
                lgb_learning_rate=study.best_params["lgb_learning_rate"],
                lgb_max_depth=study.best_params["lgb_max_depth"],
            )

            run_id = np.random.randint(1, 100000000000000)

            mae_test = np.mean(np.abs(dfp_test.p - dfp_test.y))
            rmse_test = np.sqrt(np.mean((dfp_test.p - dfp_test.y) ** 2))

            mae_val = np.mean(np.abs(dfp_val.p - dfp_val.y))
            rmse_val = np.sqrt(np.mean((dfp_val.p - dfp_val.y) ** 2))

            info.append(
                {
                    "run_id": run_id,
                    "best_params": study.best_params,
                    "location_weights": location_array_W_optim,
                    "dataset": dataset,
                    "target_col": target_col,
                    "pred_ahead": pred_ahead,
                    "mae_test": mae_test,
                    "rmse_test": rmse_test,
                    "mae_val": mae_val,
                    "rmse_val": rmse_val,
                    "dfp_test_start": str(dfp_test.index[0]),
                    "dfp_test_length": len(dfp_test.index),
                    "mae_normalized_val": (
                        np.abs((dfp_val.y - dfp_val.p))
                        / np.abs(dfp_val.original.mean())
                    ).mean(),
                    "mae_normalized_test": (
                        np.abs((dfp_test.y - dfp_test.p))
                        / np.abs(dfp_test.original.mean())
                    ).mean(),
                }
            )

            with open(info_file_name, "w+") as f:
                json.dump(info, f)
            dfp_val.reset_index().to_feather(
                "./{}/{}-validation.feather".format(result_folder, run_id)
            )
            dfp_test.reset_index().to_feather(
                "./{}/{}-test.feather".format(result_folder, run_id)
            )

/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
[I 2021-02-17 12:12:16,967] Trial 0 finished with value: 0.08117037050464962 and parameters: {'W_0': 0.207312445282263, 'W_1': 0.31094671687986464, 'W_2': 0.6423545484671814, 'W_3': 0.173007020297528, 'W_4': 0.45834592274007147, 'W_5': 0.2949217417846953, 'W_6': 0.5120511546781487, 'W_7': 0.17035153590341812, 'W_8': 0.12611870360900657, 'W_9': 0.7022994045188924, 'W_10': 0.846090559054639, 'W_11': 0.9639238202982823, 'W_12': 0.7552337736804946, 'W_13': 0.5019593664920519, 'W_14': 0.6083265437163753, 'W_15': 0.6919584012367642, 'W_16': 0.6611056271500576, 'W_17': 0.6133746882540393, 'W_18':

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


[I 2021-02-17 12:12:23,063] Trial 10 finished with value: 0.07384052880257842 and parameters: {'W_0': 0.5824809118740416, 'W_1': 0.05426108504178523, 'W_2': 0.8274785256243791, 'W_3': 0.40707008985991755, 'W_4': 0.2824400708901985, 'W_5': 0.41623197686144536, 'W_6': 0.2592385418052889, 'W_7': 0.6270788397094504, 'W_8': 0.9927812348903802, 'W_9': 0.38121954956878024, 'W_10': 0.2118453587093519, 'W_11': 0.3193169871615637, 'W_12': 0.4012574468226884, 'W_13': 0.5744561762111194, 'W_14': 0.3208147245474632, 'W_15': 0.0267660390407376, 'W_16': 0.589140755688608, 'W_17': 0.999627735431297, 'W_18': 0.08184069068145938, 'W_19': 0.9745027097121861, 'W_20': 0.8188090699500459, 'W_21': 0.026573632542725958, 'W_22': 0.9681984015621322, 'W_23': 0.45863554683109, 'W_24': 0.04858717201676277, 'W_25': 0.9845163030642017, 'extended_data': True, 'impute_missing': False, 'do_extract': True, 'shift_features': False, 'use_early_stopping': True, 'lgb_boosting_type': 'dart', 'lgb_num_leaves': 31, 'lgb_learni

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


[I 2021-02-17 12:12:56,284] Trial 89 finished with value: 0.06925177817949853 and parameters: {'W_0': 0.9620876824131486, 'W_1': 0.503097766890864, 'W_2': 0.5359057616319288, 'W_3': 0.4110809689332489, 'W_4': 0.3135203272747974, 'W_5': 0.21758350779127245, 'W_6': 0.728663880473611, 'W_7': 0.644025119791637, 'W_8': 0.680395076084522, 'W_9': 0.21037573905265486, 'W_10': 0.6175536082188936, 'W_11': 0.8244098979822806, 'W_12': 0.4238647143449631, 'W_13': 0.7847640428948967, 'W_14': 0.8557869953122121, 'W_15': 0.07212143946437155, 'W_16': 0.7326936866909427, 'W_17': 0.4334543810642252, 'W_18': 0.14675040514253984, 'W_19': 0.3925542667370117, 'W_20': 0.9103808204754098, 'W_21': 0.4435801037784055, 'W_22': 0.3265553820933715, 'W_23': 0.5380113713857594, 'W_24': 0.16158521275885496, 'W_25': 0.5043672491834209, 'extended_data': True, 'impute_missing': False, 'do_extract': True, 'shift_features': False, 'use_early_stopping': False, 'lgb_boosting_type': 'goss', 'lgb_num_leaves': 31, 'lgb_learning

...

[I 2021-02-17 12:33:32,990] Trial 4 finished with value: 0.65001727625164 and parameters: {'W_0': 0.12719914646405894, 'W_1': 0.4584529180402491, 'W_2': 0.36384412482170025, 'W_3': 0.035616112190013705, 'W_4': 0.13316660718365292, 'extended_data': True, 'impute_missing': True, 'do_extract': True, 'shift_features': True, 'use_early_stopping': False, 'lgb_boosting_type': 'goss', 'lgb_num_leaves': 31, 'lgb_learning_rate': 0.22382612988566514, 'lgb_max_depth': 8}. Best is trial 1 with value: 0.631362585608525.
[I 2021-02-17 12:33:33,305] Trial 5 finished with value: 0.7162811872451952 and parameters: {'W_0': 0.9580894433176508, 'W_1': 0.06786039771961527, 'W_2': 0.1200200032053419, 'W_3': 0.5492271511305862, 'W_4': 0.9329767214016907, 'extended_data': False, 'impute_missing': True, 'do_extract': True, 'shift_features': True, 'use_early_stopping': False, 'lgb_boosting_type': 'goss', 'lgb_num_leaves': 10, 'lgb_learning_rate': 0.057717176011512515, 'lgb_max_depth': 4}. Best is trial 1 with va

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/opt/anaconda/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))
[I 2021-02-17 12:33:33,563] Trial 6 finished with value: 0.6716180199603412 and parameters: {'W_0': 0.4601271415545223, 'W_1': 0.9959313964845034, 'W_2': 0.6971373481678199, 'W_3': 0.42537530696661385, 'W_4': 0.7561112745044697, 'extended_data': False, 'impute_missing': True, 'do_extract': False, 'shift_features': False, 'use_early_stopping': True, 'lgb_boosting_type': 'dart', 'lgb_num_leaves': 31, 'lgb_learning_rate': 0.1238678073154525, 'lgb_max_depth': 9}. Best is trial 1 with value: 0.631362585608525.
[I 2021-02-17 12:33:33,959] Trial 7 finished with value: 0.775061109708188 and parame

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


[I 2021-02-17 12:33:34,148] Trial 8 finished with value: 0.7040513498718325 and parameters: {'W_0': 0.06648836680361436, 'W_1': 0.8826061208145438, 'W_2': 0.48473920230817535, 'W_3': 0.23879912670979742, 'W_4': 0.7974313177688457, 'extended_data': False, 'impute_missing': True, 'do_extract': True, 'shift_features': False, 'use_early_stopping': False, 'lgb_boosting_type': 'gbdt', 'lgb_num_leaves': 31, 'lgb_learning_rate': 0.23747543130370163, 'lgb_max_depth': 6}. Best is trial 1 with value: 0.631362585608525.
[I 2021-02-17 12:33:34,514] Trial 9 finished with value: 0.6426033169901303 and parameters: {'W_0': 0.3600749222827504, 'W_1': 0.1616281787492697, 'W_2': 0.3845430649164878, 'W_3': 0.7586262102025791, 'W_4': 0.7526703253133997, 'extended_data': True, 'impute_missing': False, 'do_extract': False, 'shift_features': True, 'use_early_stopping': False, 'lgb_boosting_type': 'goss', 'lgb_num_leaves': 31, 'lgb_learning_rate': 0.22028798534624766, 'lgb_max_depth': 2}. Best is trial 1 with v

In [6]:
with open(info_file_name, "w+") as f:
    json.dump(info, f)

FileNotFoundError: [Errno 2] No such file or directory: './sim-res2/info.json'

In [4]:
dfp_test

,p,y,original
index_col,,,
2018-10-20,-109.311626,-109.51,-109.37
2018-10-21,-109.321621,-109.52,-109.38
2018-10-22,-109.331615,-109.53,-109.39
2018-10-23,-109.341610,-109.54,-109.40
2018-10-24,-109.351605,-109.55,-109.41
...,...,...,...
2020-06-12,-78.588033,-73.93,-78.63
2020-06-13,-78.248214,-73.60,-78.29
2020-06-14,-77.858422,-73.14,-77.90


In [8]:
-1 + 1e-6

-0.999999

In [6]:
dfp_val

,p,y,original
index_col,,,
2017-01-24,-0.0,-103.169905,-103.03
2017-01-25,-0.0,-103.179905,-103.04
2017-01-28,-0.0,-103.209905,-103.07
2017-01-29,-0.0,-103.219905,-103.08
2017-01-30,-0.0,-103.229905,-103.09
...,...,...,...
2018-09-30,-0.0,-109.309934,-109.19
2018-10-01,-0.0,-109.319934,-109.20
2018-10-02,-0.0,-109.329934,-109.21


In [42]:
dfp_val.reset_index().to_feather("./sim-res/{}-validation.feather".format(run_id))
dfp_test.reset_index().to_feather("./sim-res/{}-test.feather".format(run_id))

# info.append({
#     'run_id': run_id,
#     'best_params': best_params,
#     'location_weights': location_array_W_optim,
#     'dataset': dataset,
#     'target_col': target_col,
#     'pred_ahead': pred_ahead
# })
# with open(info_file_name, 'w') as f:
#     json.dump(info, f)